In [6]:
import json
import pprint
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import pytextrank
import statistics
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger
from nltk.corpus import stopwords
import os

java_path = "C:\Program Files (x86)\Common Files\Oracle\Java\javapath\java.exe"
os.environ['JAVAHOME'] = java_path
path_to_model = "C:\\Users\\sathishm\\Documents\\DSE\\DSE 203 ETL\\Final Proj\\stanford-postagger-full-2018-10-16\\stanford-postagger-full-2018-10-16\\models\\english-bidirectional-distsim.tagger"
path_to_jar = "C:\\Users\\sathishm\\Documents\\DSE\\DSE 203 ETL\\Final Proj\\stanford-postagger-full-2018-10-16\\stanford-postagger-full-2018-10-16\\stanford-postagger.jar"
tagger = StanfordPOSTagger(path_to_model, path_to_jar)
tagger.java_options = '-mx4096m'  ### Setting higher memory limit for long sentences

# A function to split the data into tokens
def tokenize_text(text, sublist):
    return_sent = []
    tokens = []
    sentences = tokenize.sent_tokenize(text)  # Break Sentences
    for s in sentences:  # Pick only those sentences that has the keyword
        #print(s)
        if any(kw in s for kw in sublist):
            return_sent.append(s)
    return return_sent

def remove_stopwords(tokens):
    # Remove the stop words by using the english stop words provided by NLTK
    e_stopwords = set(stopwords.words('english'))
    return_sent = []
    for s in tokens:
        text = ' '.join([word for word in s.split() if len(word.lower()) > 1 and (word.lower() not in e_stopwords)])
        return_sent.append(text)
    return return_sent

tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

skip_f = open('skiplist2.txt', "r")
skip_list = []
for line in skip_f:
    skip_list.append(line.strip())

#print(skip_list)

with open('updated_scraped_data.json') as f:
    data = json.load(f)

maindict = {}
for companykey in data.keys():
    if (companykey not in skip_list):# and ('Drawbridge' in companykey):
        print ('##################'+companykey)
        IBM_json = data[companykey]
        for key in IBM_json.keys():
            if '[edit]' in key or 'Intro' in key:
                if 'See also' in key or 'References' in key or 'External links' in key:
                    pass
                else:
                    if 'Intro' in key:
                        IBM_json[key] = [IBM_json[key]]
                    print("**********" + key)
                    for item in IBM_json[key]:
                        if isinstance(item, str):
                            item_ = item.replace("\n", "")
                            regex = '\[.*?\]'
                            item_ = re.sub(regex, '', item_)
                            item1 = re.sub(regex, '', item)
                            doc = nlp(item_)
                            rank = []
                            text = []
                            for p in doc._.phrases:
                                rank.append(p.rank)
                                text.append(p.text)
                            temp = dict(zip(rank, text))
                            sublist = []
                            if len(temp.keys()) > 15:
                                midrank = statistics.median(temp.keys())
                            else:
                                midrank = 0
                            for subkey in sorted(temp.keys(), reverse=True):
                                if (subkey > midrank):
                                    sam = temp[subkey]
                                    sublist.append(sam)
                                else:
                                    break
                    IBM_json[key].append(sublist)
                    tokens = tokenize_text(item1, sublist)
                    clean_tokens = remove_stopwords(tokens)
                    sdict = {}
                    count = 0
                    for sentence in clean_tokens:
                        temp = tagger.tag(word_tokenize(sentence))
                        if any('VB' in item[1] for item in temp):
                            if any('NN' in item[1] for item in temp):
                                sdict[count] = ''.join(str(tuple(x)) for x in temp)
                                count = count + 1
                    IBM_json[key].append(sdict)
            maindict[companykey] = IBM_json

with open('upd_json__.json', "w+") as f:
    f.write(json.dumps(maindict, indent=4))

##################LinkedIn Pulse
**********History[edit]
**********Intro
##################Drawbridge (company)
**********History[edit]
**********Technology[edit]
**********Reception[edit]
**********Financials[edit]
**********Intro
##################Glassdoor
**********Founding[edit]
**********Website[edit]
**********Reports[edit]
**********Financing[edit]
**********Awards[edit]
**********Intro
##################Bright.com
**********Intro
##################SlideShare
**********History[edit]
**********Zipcasts[edit]
**********Intro
##################LinkedIn
**********Company overview[edit]
**********History[edit]
**********Membership[edit]
**********Future plans[edit]
**********New user interface in 2017[edit]
**********Discontinued features[edit]
**********Business units[edit]
**********Reception[edit]
**********International restrictions[edit]
**********SNA LinkedIn[edit]
**********Surveillance and NSA program[edit]
**********Criticism[edit]
**********Science[edit]
**********Intro
##

**********Research[edit]
**********Applications[edit]
**********Intro
##################Lionhead Studios
**********History[edit]
**********Games[edit]
**********Intro
##################LinkExchange
**********Intro
##################Jellyfish.com
**********History[edit]
**********Live Search Cashback[edit]
**********Intro
##################Groove Networks
**********Intro
##################GreenButton
**********History[edit]
**********Intro
##################GitHub
**********History[edit]
**********Company affairs[edit]
**********Services[edit]
**********Sanctions[edit]
**********Developed projects[edit]
**********Censorship[edit]
**********Prominent users[edit]
**********Intro
##################Windows Defender
**********Basic features[edit]
**********History[edit]
**********Advanced features[edit]
**********Windows Vista-specific functionality[edit]
**********Windows Defender Offline[edit]
**********Mitigated security vulnerability[edit]
**********Reviews[edit]
**********Intro
########

**********Honors and awards[edit]
**********Intro
##################OStatus
**********History[edit]
**********Standards work[edit]
**********Projects using OStatus[edit]
**********Intro
##################John W. Stanton
**********Family[edit]
**********Intro
##################Helmut Panke
**********Education[edit]
**********Career[edit]
**********Current mandates[edit]
**********Intro
##################Charles Noski
**********Education[edit]
**********Career[edit]
**********Intro
##################John W. Thompson
**********Personal life[edit]
**********Professional life[edit]
**********Political activity[edit]
**********Intro
##################Paul Allen
**********Early life[edit]
**********Microsoft[edit]
**********Business and investments[edit]
**********Philanthropy[edit]
**********Personal life[edit]
**********Death[edit]
**********Awards and recognition[edit]
**********Idea Man[edit]
**********Further reading[edit]
**********Intro
################## (Cork newspaper)
**********His

**********Architectures[edit]
**********Applications[edit]
**********Examples[edit]
**********Theoretical foundations[edit]
**********Notes[edit]
**********Further reading[edit]
**********Intro
##################Pinyin
**********History[edit]
**********Initials and finals[edit]
**********Tones[edit]
**********Orthographic rules[edit]
**********Comparison with other orthographies[edit]
**********Usage[edit]
**********Notes[edit]
**********Further reading[edit]
**********Intro
##################Shiftgig
**********Overview[edit]
**********Company[edit]
**********Intro
##################TechRepublic
**********Intro
##################Trade magazine
**********Intro
##################eConozco
**********Intro
##################Capazoo
**********Site features[edit]
**********Partnerships[edit]
**********Intro
##################Bolt (website)
**********1996–2006[edit]
**********2006–2009[edit]
**********Bolt and American Idol[edit]
**********Intro
##################Avatars United
**********Featu

##################Snow (app)
**********Background[edit]
**********Features[edit]
**********Intro
##################Promo.com
**********History[edit]
**********Business[edit]
**********Intro
##################Renren
**********History[edit]
**********Corporate affairs[edit]
**********Open platform[edit]
**********Privacy policy[edit]
**********Functions[edit]
**********Kaixin[edit]
**********Footnotes[edit]
**********Intro
##################Readgeek
**********Notes[edit]
**********Intro
##################Qzone
**********History[edit]
**********Some Features[edit]
**********Versions and Key Improvements[edit]
**********Censorship[edit]
**********Profits[edit]
**********Intro
##################Plurk
**********History[edit]
**********Features and technology[edit]
**********Availability in other languages[edit]
**********Reception[edit]
**********Controversies[edit]
**********Intro
##################Snapchat
**********Intro
##################Pixnet
**********History[edit]
**********Intro
###

**********Intro
##################Facebook
**********Intro
##################Beamer (LaTeX)
**********Method[edit]
**********Output options[edit]
**********Dependencies[edit]
**********Further reading[edit]
**********Intro
##################Crunchbase
**********History[edit]
**********Products[edit]
**********User base[edit]
**********Intro
##################SMA Solar Technology
**********Company profile[edit]
**********History[edit]
**********Products[edit]
**********Stock[edit]
**********Awards[edit]
**********Intro
##################Sartorius AG
**********Company Profile[edit]
**********Early history[edit]
**********Recent history (since 2000)[edit]
**********Products & services[edit]
**********Awards[edit]
**********Intro
##################Qiagen
**********History[edit]
**********Products[edit]
**********Application areas[edit]
**********Management[edit]
**********Structure of the company[edit]
**********Intro
##################QSC AG
**********Intro
##################Pfeiffer Vacu

**********History[edit]
**********Office locations[edit]
**********Accolades[edit]
**********Notes[edit]
**********Intro
##################Recruit (company)
**********History[edit]
**********Overview[edit]
**********Intro
##################Tim Besse
**********Intro
##################São Paulo
**********History[edit]
**********Geography[edit]
**********Demographics[edit]
**********Government[edit]
**********Economy[edit]
**********Urban infrastructure[edit]
**********Education[edit]
**********Health care[edit]
**********Transport[edit]
**********Culture[edit]
**********Sports[edit]
**********Intro
##################Rich Barton
**********Early life[edit]
**********Entrepreneurship[edit]
**********Intro
##################Robert Hohman
**********Intro
##################BoardEx
**********History[edit]
**********Features[edit]
**********Name[edit]
**********Locations[edit]
**********Sites with comparable features[edit]
**********Intro
##################Hubert Burda Media
**********History[ed